In [2]:
# 놀라운 점, 노원구가 공원 개수는 제일 적은데, 면적은 제일 넓음 !!!

import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64
import seaborn as sns

from dotenv import load_dotenv
import os
import json

# .env 파일을 찾아 환경 변수로 로드
load_dotenv("/home/jiming/dev_ws/eda-repo-3/env")

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")


In [ ]:
# 차치구 공원 총 면적

# ▶️ 전국 도시 공원 지역 정보 가져오기
# 서울 관련 정보만 가져오기
query = """
SELECT park_area as 면적, provider_name as 제공처
FROM korea_urban_park_info
WHERE jibun_address like "%%서울%%"
"""
df = pd.read_sql(query, engine)

df.columns =['면적', '제공처']


# 자치구별로 구분하기
df['자치구'] = df['제공처'].str.extract(r'서울특별시\s+(\S+구)')
df=df.drop(columns=['제공처'])
df.columns =['면적', '자치구']

# 자치구 별로 공원 총 면적 합치기
df_grouped=df.groupby('자치구')['면적'].sum().reset_index()
df_grouped.columns = ['자치구', '공원 총 면적']

# print(df_grouped)

df_sorted = df_grouped.sort_values(by='공원 총 면적', ascending=False)
df_sorted.columns = ['자치구', '공원 총 면적']
print(df_sorted)


            면적  자치구
0       1945.0  강서구
1       1494.0  강서구
2        864.0  강서구
3       2000.0  강서구
4       2066.0  강서구
...        ...  ...
1748  200327.0  송파구
1749   42448.0  송파구
1750   10741.0  송파구
1751   10569.0  송파구
1752    1617.7  송파구

[1753 rows x 2 columns]
     자치구      공원 총 면적
8    노원구  13235453.00
9    도봉구   9915333.00
17   송파구   3561293.30
3    강서구   3275303.20
11   동작구   2520624.60
18   양천구   2258307.10
21   은평구   2120125.20
0    강남구   2032363.00
1    강동구   1934993.30
22   종로구   1811233.10
2    강북구   1663911.80
24   중랑구   1521654.10
13  서대문구   1370712.10
6    구로구   1285616.99
16   성북구   1257497.20
14   서초구   1155965.00
12   마포구   1106624.90
10  동대문구    762904.20
20   용산구    503097.60
4    관악구    332109.00
19  영등포구    272090.50
23    중구    271448.90
15   성동구    179343.96
7    금천구    146089.30
5    광진구     48530.20


In [4]:
# 자치구 공원 수

# ▶️ 전국 도시 공원 지역 정보 가져오기
# 서울 관련 정보만 가져오기
query = """
SELECT jibun_address as 주소, latitude as 위도, longitude as 경도, park_area as 면적, provider_name as 제공처
FROM korea_urban_park_info
WHERE jibun_address like "%%서울%%"
"""
df = pd.read_sql(query, engine)

df.columns =['주소', '위도', '경도', '면적', '제공처']

# 자치구별로 구분하기
df['자치구'] = df['제공처'].str.extract(r'서울특별시\s+(\S+구)')


df_grouped = df.groupby('자치구').size().reset_index(name ='count')

df_sorted = df_grouped.sort_values(by='count', ascending=False)
df_sorted.columns = ['자치구', '공원 수']
print(df_sorted)


     자치구  공원 수
3    강서구   151
17   송파구   140
0    강남구   133
14   서초구   128
21   은평구   101
18   양천구    96
12   마포구    90
1    강동구    74
9    도봉구    71
6    구로구    62
4    관악구    60
24   중랑구    58
13  서대문구    55
2    강북구    54
16   성북구    53
7    금천구    52
11   동작구    52
19  영등포구    50
10  동대문구    50
20   용산구    48
15   성동구    47
22   종로구    43
5    광진구    43
23    중구    38
8    노원구     4
